In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from scipy import signal
from scipy.io import loadmat
from sklearn.metrics import confusion_matrix
import os
from tensorflow.keras.models import Sequential, Model, load_model
import datetime
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras as K
from tqdm import tqdm
from sklearn.decomposition import PCA
import scipy as sp

C:\Users\mukun\anaconda3\envs\blurproject\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (5.1.0)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [2]:
def rms(data):
    return np.sqrt(np.mean(data ** 2))


def hist(data, nbins=20):
    histsig, bin_edges = np.histogram(data, bins=nbins)
    return tuple(histsig)


def entropy(data):
    pk = sp.stats.rv_histogram(np.histogram(data, bins=20)).pdf(data)
    return sp.stats.entropy(pk)


def kurtosis(data):
    return sp.stats.kurtosis(data)


def zero_cross(data):
    return len(np.where(np.diff(np.sign(data)))[0]) / len(data)


def min(data):
    return np.min(data)


def max(data):
    return np.max(data)


def mean(data):
    return np.mean(data)


def median(data):
    return np.median(data)


def fft(data):
    return np.fft.fft(data)


def psd(data):
    return np.abs(np.fft.fft(data)) ** 2

In [3]:
def get_data(path, file):
    mat = loadmat(os.path.join(path, file))
    data = pd.DataFrame(mat['emg'])
    data['stimulus'] = mat['restimulus']
    data['repetition'] = mat['repetition']

    return data


def normalise(data, train_reps):
    x = [np.where(data.values[:, 13] == rep) for rep in train_reps]
    indices = np.squeeze(np.concatenate(x, axis=-1))
    train_data = data.iloc[indices, :]
    train_data = data.reset_index(drop=True)

    scaler = StandardScaler(with_mean=True,
                            with_std=True,
                            copy=False).fit(train_data.iloc[:, :12])

    scaled = scaler.transform(data.iloc[:, :12])
    normalised = pd.DataFrame(scaled)
    normalised['stimulus'] = data['stimulus']
    normalised['repetition'] = data['repetition']
    return normalised


def filter_data(data, f, butterworth_order=4, btype='lowpass'):
    emg_data = data.values[:, :12]

    f_sampling = 2000
    nyquist = f_sampling / 2
    if isinstance(f, int):
        fc = f / nyquist
    else:
        fc = list(f)
        for i in range(len(f)):
            fc[i] = fc[i] / nyquist

    b, a = signal.butter(butterworth_order, fc, btype=btype)
    transpose = emg_data.T.copy()

    for i in range(len(transpose)):
        transpose[i] = (signal.lfilter(b, a, transpose[i]))

    filtered = pd.DataFrame(transpose.T)
    filtered['stimulus'] = data['stimulus']
    filtered['repetition'] = data['repetition']

    return filtered


def rectify(data):
    return abs(data)


def windowing(data, reps, gestures, win_len, win_stride):
    if reps:
        x = [np.where(data.values[:, 13] == rep) for rep in reps]
        indices = np.squeeze(np.concatenate(x, axis=-1))
        data = data.iloc[indices, :]
        data = data.reset_index(drop=True)

    if gestures:
        x = [np.where(data.values[:, 12] == move) for move in gestures]
        indices = np.squeeze(np.concatenate(x, axis=-1))
        data = data.iloc[indices, :]
        data = data.reset_index(drop=True)

    idx = [i for i in range(win_len, len(data), win_stride)]

    X = np.zeros([len(idx), win_len, len(data.columns) - 2])
    y = np.zeros([len(idx), ])
    reps = np.zeros([len(idx), ])

    for i, end in enumerate(idx):
        start = end - win_len
        X[i] = data.iloc[start:end, 0:12].values
        y[i] = data.iloc[end, 12]
        reps[i] = data.iloc[end, 13]

    return X, y, reps

In [4]:
data = get_data('C:/Users/mukun/Desktop/IITKGP Internship/Data/s1/s1','S1_E1_A1.mat')

In [5]:
emg_low = filter_data(data=data, f=20, butterworth_order=4, btype='lowpass')

emg_band = filter_data(data=data, f=(20,40), butterworth_order=4, btype='bandpass')

emg_high = filter_data(data=data, f=20, butterworth_order=4, btype='high')

In [6]:
rms = rms(data)

C:\Users\mukun\anaconda3\envs\blurproject\lib\site-packages\numpy\core\fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [7]:
print(rms)

0             1079.025312
1             1280.023437
2              221.152821
3              176.900175
4              414.939491
5              446.620767
6              330.178231
7              641.214156
8              436.507929
9             1521.847726
10             251.920630
11             192.258229
stimulus         5.692128
repetition       3.090025
dtype: float64


In [8]:
average_rms = np.mean(rms)
print("Average RMS: ", average_rms)

Average RMS:  500.0979326561757


In [9]:
hist = hist(data, nbins=20)
hist

(10,
 21,
 58,
 219,
 792,
 2858,
 11147,
 63983,
 1737743,
 15407866,
 102854,
 17476,
 4459,
 1275,
 420,
 130,
 29,
 13,
 5,
 4)

In [10]:
average_hist = np.mean(hist)
print("Average hist: ", hist)

Average hist:  (10, 21, 58, 219, 792, 2858, 11147, 63983, 1737743, 15407866, 102854, 17476, 4459, 1275, 420, 130, 29, 13, 5, 4)


In [12]:
import numpy as np

def histogram_mean(histogram):
    bin_values = np.arange(len(histogram))
    total_weight = np.sum(histogram)

    mean = np.sum(bin_values * histogram) / total_weight

    return mean

In [13]:
histogram_mean(hist)

8.898741954666152

In [14]:
entropy = entropy(data)
entropy

array([13.84129557, 13.86535153, 13.99138173, 14.0030338 , 13.95609243,
       13.95268138, 13.95926203, 13.90753146, 13.96401531, 13.8123159 ,
       13.99525945, 13.98962215, 14.03012423, 14.03012423])

In [15]:
average_entropy = np.mean(entropy)
print("Average entropy: ", average_entropy)

Average entropy:  13.949863658360576


In [16]:
kurtosis = kurtosis(data)
kurtosis

array([14.574699 , 25.849886 , 13.950155 , 15.019121 , 13.473644 ,
       14.060223 , 15.914549 , 14.253283 , 23.664936 , 18.986076 ,
       55.727867 ,  3.4716606, -1.1060382, -1.2600695], dtype=float32)

In [17]:
average_kurtosis = np.mean(kurtosis)
print("Average kurtosis: ", average_kurtosis)

Average kurtosis:  16.184286


In [18]:
zero_cross(data)

6.3375962071450065

In [19]:
min = min(data)
min

C:\Users\mukun\anaconda3\envs\blurproject\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)


0            -12303.000000
1            -23873.000000
2             -3302.000000
3             -3945.000000
4             -7189.000000
5             -9552.000000
6             -8451.000000
7             -9467.000000
8             -8606.000000
9            -25167.000000
10            -8414.222656
11            -2437.400879
stimulus          0.000000
repetition        0.000000
dtype: float32

In [20]:
average_min = np.mean(min)
print("Average min: ", average_min)

Average min:  -8764.759


In [21]:
max = max(data)
max

C:\Users\mukun\anaconda3\envs\blurproject\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


0             15586.000000
1             20316.000000
2              3661.761963
3              3176.000000
4              6295.000000
5              6311.000000
6              7914.000000
7             11528.000000
8              8389.000000
9             30114.000000
10             5896.604492
11             2027.485840
stimulus         12.000000
repetition        6.000000
dtype: float32

In [22]:
average_max = np.mean(max)
print("Average max:", average_max)

Average max: 8659.489


In [23]:
mean = mean(data)
mean

C:\Users\mukun\anaconda3\envs\blurproject\lib\site-packages\numpy\core\fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


0            -11.579947
1             -5.233460
2             -5.976496
3             -4.945025
4             -2.861733
5             -6.374892
6             -7.428028
7             -8.064356
8              7.245286
9             -6.349988
10            -5.138816
11            -9.080110
stimulus       3.867075
repetition     2.203224
dtype: float64

In [24]:
average_mean = np.mean(mean)
print("Average mean: ", average_mean)

Average mean:  -4.265518919532055


In [25]:
median(data)

0.0

In [26]:
fft = fft(data)
fft

array([[  37.89604282  +0.j        , -308.07955796 -77.38963482j,
        -169.40842043+130.68313805j, ...,  128.06045478 -15.72235314j,
        -169.40842043-130.68313805j, -308.07955796 +77.38963482j],
       [-147.33532906  +0.j        ,  -30.6868456  -91.95574143j,
         -60.07461409 +33.89180305j, ...,   -4.53563699 +58.93881399j,
         -60.07461409 -33.89180305j,  -30.6868456  +91.95574143j],
       [ -37.83737779  +0.j        ,   37.26974025 -91.24068332j,
        -174.72470341 +19.9349212j , ...,  143.70072798 +21.70501805j,
        -174.72470341 -19.9349212j ,   37.26974025 +91.24068332j],
       ...,
       [ 892.01028347  +0.j        , -568.96734786+269.25110528j,
        -260.3229398 -210.58081666j, ..., 1097.77685459 -70.24854381j,
        -260.3229398 +210.58081666j, -568.96734786-269.25110528j],
       [ 637.79352427  +0.j        , -370.59136851  +9.91177164j,
        -224.1322417 -209.20042215j, ...,  759.06765295-157.16569945j,
        -224.1322417 +209.20042215j

In [27]:
average_fft = np.mean(fft)
print("Average FFT: ", average_fft)

Average FFT:  (-11.579951687685146-7.000200757829533e-17j)


In [28]:
psd = psd(data)
psd

array([[   1436.1100617 ,  100902.16960962,   45777.29548184, ...,
          16646.67246746,   45777.29548184,  100902.16960962],
       [  21707.69918798,    9397.54087553,    4757.61357236, ...,
           3494.35579708,    4757.61357236,    9397.54087553],
       [   1431.66715777,    9713.89583088,   30926.12306589, ...,
          21121.0070302 ,   30926.12306589,    9713.89583088],
       ...,
       [ 795682.3458165 ,  396220.0006236 ,  112112.31332799, ...,
        1210048.88039196,  112112.31332799,  396220.0006236 ],
       [ 406780.57959474,  137436.20562985,   94000.07839843, ...,
         600884.75883849,   94000.07839843,  137436.20562985],
       [ 156167.23339494,  149388.13033648,   43849.88197268, ...,
         314765.40357645,   43849.88197268,  149388.13033648]])

In [29]:
average_psd = np.mean(psd)
print("Average PSD: ", average_psd)

Average PSD:  6381804.841468112


# Spectral domain features

In [32]:
def spectral_roll_off(data, roll_percent):
    # Calculate the power spectral density (PSD)
    power_spectrum = psd

    # Sort the PSD values in descending order
    sorted_spectrum = np.sort(power_spectrum)[::-1]

    # Calculate the cumulative sum of the sorted PSD values
    cumulative_sum = np.cumsum(sorted_spectrum)

    # Find the index where the cumulative sum exceeds the roll-off percentage
    roll_index = np.argmax(cumulative_sum >= (roll_percent / 100) * np.sum(power_spectrum))

    # Calculate the roll-off frequency
    sampling_rate = 1  # Modify this if your data has a specific sampling rate
    roll_frequency = (roll_index / len(power_spectrum)) * (sampling_rate / 2)

    return roll_frequency

In [33]:
roll_off_point = spectral_roll_off(data, roll_percent=95)
print("Spectral Roll-off Point:", roll_off_point)

Spectral Roll-off Point: 6.350126635592065


In [39]:
def spectral_flatness(data):
    # Calculate the power spectral density (PSD)
    power_spectrum = psd

    # Calculate the geometric mean of the PSD values
    geometric_mean = sp.stats.gmean(power_spectrum)

    # Calculate the arithmetic mean of the PSD values
    arithmetic_mean = np.mean(power_spectrum)

    # Calculate the spectral flatness
    flatness = 10 * np.log10(geometric_mean / arithmetic_mean)

    return flatness

In [41]:
flatness_value = spectral_flatness(data)
print("Spectral Flatness:", flatness_value)
average_flatness_value = np.mean(flatness_value)
print("Average Spectral Flatness: ", average_flatness_value)

Spectral Flatness: [-14.23046312 -11.5150677  -11.21566952 -10.97471299 -10.64486511
 -11.14622194 -10.23314107 -14.20675717 -10.23314107 -11.14622194
 -10.64486511 -10.97471299 -11.21566952 -11.5150677 ]
Average Spectral Flatness:  -11.421184068074831


In [42]:
def spectral_crust(data, crust_percent):
    # Calculate the power spectral density (PSD)
    power_spectrum = psd

    # Sort the PSD values in descending order
    sorted_spectrum = np.sort(power_spectrum)[::-1]

    # Calculate the cumulative sum of the sorted PSD values
    cumulative_sum = np.cumsum(sorted_spectrum)

    # Normalize the cumulative sum
    cumulative_distribution = cumulative_sum / np.sum(power_spectrum)

    # Find the index where the cumulative distribution exceeds the crust percentage
    crust_index = np.argmax(cumulative_distribution >= (crust_percent / 100))

    # Calculate the crust frequency
    sampling_rate = 1  # Modify this if your data has a specific sampling rate
    crust_frequency = (crust_index / len(power_spectrum)) * (sampling_rate / 2)

    return crust_frequency

In [43]:
crust_value = spectral_crust(data, crust_percent=85)
print("Spectral Crust:", crust_value)

Spectral Crust: 5.4001478961709175


In [44]:
def spectral_decrease(data):
    # Calculate the power spectral density (PSD)
    power_spectrum = psd

    # Take the logarithm (base 10) of the PSD values
    log_power_spectrum = np.log10(power_spectrum)

    # Calculate the discrete derivative of the logarithmically transformed PSD
    derivative = np.diff(log_power_spectrum)

    # Calculate the average of the derivative values
    average_derivative = np.mean(derivative)

    # Calculate the spectral decrease (negative of the average derivative)
    spectral_decrease = -average_derivative

    return spectral_decrease

In [45]:
decrease_value = spectral_decrease(data)
print("Spectral Decrease:", decrease_value)

Spectral Decrease: -0.020887657109746802


In [66]:
import numpy as np
from sklearn.linear_model import LinearRegression

def spectral_slope(data):
    # Calculate the Fourier transform of the data
    fft_data = np.fft.fft(data)

    # Calculate the power spectral density (PSD)
    power_spectrum = np.abs(fft_data) ** 2

    # Get the frequencies corresponding to the PSD values
    frequencies = np.fft.fftfreq(len(data))

    # Reshape the frequencies array for linear regression
    frequencies = frequencies.reshape(-1, 1)

    # Fit linear regression model
    model = LinearRegression()
    model.fit(frequencies, power_spectrum)

    # Extract the slope coefficient
    slope_coefficient = model.coef_[0]

    return slope_coefficient

In [67]:
slope_value = spectral_slope(data)
print("Spectral Slope:", slope_value)

Spectral Slope: [2523226.53452005]


In [76]:
import numpy as np
from sklearn.linear_model import LinearRegression

def spectral_spread(data):
    # Calculate the power spectral density (PSD)
    power_spectrum = np.abs(np.fft.fft(data)) ** 2

    # Get the frequencies corresponding to the PSD values
    frequencies = np.fft.fftfreq(len(data))

    # Reshape the frequencies array for linear regression
    frequencies = frequencies.reshape(-1, 1)

    # Fit linear regression model
    model = LinearRegression()
    model.fit(frequencies, power_spectrum)

    # Calculate the residuals
    residuals = power_spectrum - model.predict(frequencies)

    # Calculate the spectral spread
    spectral_spread = np.sqrt(np.sum(residuals ** 2 * frequencies) / np.sum(power_spectrum))

    return spectral_spread

In [77]:
spread_value = spectral_spread(data)
print("Spectral Spread:", spread_value)

Spectral Spread: 4933.006817993984
